In [26]:
import pandas as pd
import numpy as np
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.Dataset import load_dataset

In [ ]:
data, labels = load_dataset("boston_housing")

In [29]:
data

array([[6.3000e-03, 1.8000e+01, 2.3100e+00, ..., 3.9690e+02, 4.9800e+00,
        2.4000e+01],
       [2.7300e-02, 0.0000e+00, 7.0700e+00, ..., 3.9690e+02, 9.1400e+00,
        2.1600e+01],
       [2.7300e-02, 0.0000e+00, 7.0700e+00, ..., 3.9283e+02, 4.0300e+00,
        3.4700e+01],
       ...,
       [6.0800e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 5.6400e+00,
        2.3900e+01],
       [1.0960e-01, 0.0000e+00, 1.1930e+01, ..., 3.9345e+02, 6.4800e+00,
        2.2000e+01],
       [4.7400e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 7.8800e+00,
        1.1900e+01]], shape=(506, 14))

In [28]:
g, edges = fci(data)
pdy = GraphUtils.to_pydot(g)
pdy.write_png('boston_housing.png')

Depth=0, working on node 13: 100%|██████████| 14/14 [00:00<00:00, 470.90it/s]


X9 --> X1
X1 --> X12
X5 --> X3
X3 --> X10
X7 --> X5
X14 --> X6
X9 --> X10
X14 --> X11
X13 --> X14
